In [1]:
import os
import math
import re
import collections
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from regressors import stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import *--+

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.cluster import KMeans
from sklearn.metrics import hamming_loss
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabaz_score
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')

<font size = 5>
(a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics. uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.

In [3]:
df = pd.read_csv('Frogs_MFCCs.csv')

train_df, test_df = train_test_split(df, train_size=0.7, test_size = 0.3)

X_train = train_df.iloc[:,:-4]
y_train = train_df.iloc[:,-4:-1]
X_test = test_df.iloc[:,:-4]
y_test = test_df.iloc[:,-4:-1]

X_train.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22
2210,1.0,0.142665,0.182539,0.466349,0.088505,-0.011861,-0.066383,0.105979,0.305904,0.048669,...,0.245476,-0.164588,-0.180686,0.125307,0.111628,-0.017731,-0.050419,-0.067309,0.057707,0.121706
1792,1.0,0.299430,0.205311,0.590564,0.174769,0.013744,-0.139228,0.003150,0.273798,0.092782,...,0.438382,0.022074,-0.261175,0.020437,0.197870,0.042685,-0.099221,-0.134869,-0.023347,0.146671
732,1.0,0.183875,-0.004147,0.136533,0.105952,0.148560,0.146741,-0.022179,-0.120005,0.006283,...,-0.004177,0.085480,-0.009452,-0.097577,0.008150,0.058082,-0.022958,-0.056031,-0.013966,-0.003480
5010,1.0,0.616106,0.703758,0.472784,-0.029115,0.056836,0.006603,0.182854,0.222200,-0.261834,...,-0.160116,-0.277389,0.243428,0.092666,-0.153912,-0.033288,0.026084,0.134206,0.082594,-0.161210
4044,1.0,0.390029,0.214202,0.598590,0.247795,0.070456,-0.161894,0.069990,0.281743,0.042360,...,0.289659,-0.118219,-0.245198,0.065839,0.243514,0.068396,-0.061918,-0.188621,0.017854,0.188410


In [5]:
X_train.count()

MFCCs_ 1    5036
MFCCs_ 2    5036
MFCCs_ 3    5036
MFCCs_ 4    5036
MFCCs_ 5    5036
MFCCs_ 6    5036
MFCCs_ 7    5036
MFCCs_ 8    5036
MFCCs_ 9    5036
MFCCs_10    5036
MFCCs_11    5036
MFCCs_12    5036
MFCCs_13    5036
MFCCs_14    5036
MFCCs_15    5036
MFCCs_16    5036
MFCCs_17    5036
MFCCs_18    5036
MFCCs_19    5036
MFCCs_20    5036
MFCCs_21    5036
MFCCs_22    5036
dtype: int64

In [6]:
X_test.count()

MFCCs_ 1    2159
MFCCs_ 2    2159
MFCCs_ 3    2159
MFCCs_ 4    2159
MFCCs_ 5    2159
MFCCs_ 6    2159
MFCCs_ 7    2159
MFCCs_ 8    2159
MFCCs_ 9    2159
MFCCs_10    2159
MFCCs_11    2159
MFCCs_12    2159
MFCCs_13    2159
MFCCs_14    2159
MFCCs_15    2159
MFCCs_16    2159
MFCCs_17    2159
MFCCs_18    2159
MFCCs_19    2159
MFCCs_20    2159
MFCCs_21    2159
MFCCs_22    2159
dtype: int64

In [14]:
print('2159/7195=%f, \n about 30 percent of the data is in the Test set and the 70 percent is in the Training set'%(2159/7195))

2159/7195=0.300069, 
 about 30 percent of the data is in the Test set and the 70 percent is in the Training set


<font size = 5> (b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-class classification is to train a classifier for each label. We first try this approach:

i. Research exact match and hamming score/ loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem.

## Exact match (also called Subset accuracy): is the most strict metric, indicating the percentage of samples that have all their labels classified correctly. The disadvantage of this measure is that multi-class classification problems have a chance of being partially correct, but here we ignore those partially correct matches.

## Hamming-Loss (Example based measure): In simplest of terms, Hamming-Loss is the fraction of labels that are incorrectly predicted, i.e., the fraction of the wrong labels to the total number of labels. The method choose in hamming loss was to give each label equal weight.

## In general, there is no magical metric that is the best for every problem. In every problem you have different needs, and you should optimize for them.

sources: https://stats.stackexchange.com/questions/336820/what-is-a-hamming-loss-will-we-consider-it-for-an-imbalanced-binary-classifier https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff

<font size= 5> ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation.1 You are welcome to try to solve the problem with both standardized and raw attributes and report the results.

In [27]:
# family 
y_train_family = y_train['Family']
Cs = np.linspace(0.001,10000,10)
gammas = np.linspace(0.001,3,10)
param_set = {'C': Cs, 'gamma' : gammas}
grid_search_cv = GridSearchCV(SVC(kernel='rbf'), param_set , cv=10)
grid_search_cv.fit(X_train, y_train_family)
best_param = grid_search_cv.best_params_
print('Best Parameters: C = %f, Gamma = %f'%(best_param['C'],best_param['gamma']))


Best Parameters: C = 1111.112000, Gamma = 1.333889


In [30]:
pred = pd.DataFrame()
pred['Family_1'] = grid_search_cv.predict(X_test)

acc_score = accuracy_score(y_test['Family'], pred['Family_1'])
hamming_l = hamming_loss(y_test['Family'], pred['Family_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9898100972672533
Hamming Loss: 0.010189902732746642


In [31]:
# Genus
y_train_genus = y_train['Genus']
Cs = np.linspace(0.001,10000,10)
gammas = np.linspace(0.001,3,10)
param_set = {'C': Cs, 'gamma' : gammas}
grid_search_cv = GridSearchCV(SVC(kernel='rbf'), param_set , cv=10)
grid_search_cv.fit(X_train, y_train_genus)
best_param = grid_search_cv.best_params_
print('Best Parameters: C = %f, Gamma = %f'%(best_param['C'],best_param['gamma']))


Best Parameters: C = 1111.112000, Gamma = 1.333889


In [32]:
pred['Genus_1'] = grid_search_cv.predict(X_test)

acc_score = accuracy_score(y_test['Genus'], pred['Genus_1'])
hamming_l = hamming_loss(y_test['Genus'], pred['Genus_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9884205650764243
Hamming Loss: 0.01157943492357573


In [33]:
# Species
y_train_species = y_train['Species']
Cs = np.linspace(0.001,10000,10)
gammas = np.linspace(0.001,3,10)
param_set = {'C': Cs, 'gamma' : gammas}
grid_search_cv = GridSearchCV(SVC(kernel='rbf'), param_set , cv=10)
grid_search_cv.fit(X_train, y_train_species)
best_param = grid_search_cv.best_params_
print('Best Parameters: C = %f, Gamma = %f'%(best_param['C'],best_param['gamma']))


Best Parameters: C = 1111.112000, Gamma = 2.000333


In [34]:
pred['Species_1'] = grid_search_cv.predict(X_test)

acc_score = accuracy_score(y_test['Species'], pred['Species_1'])
hamming_l = hamming_loss(y_test['Species'], pred['Species_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9888837424733673
Hamming Loss: 0.0111162575266327


<font size = 5>iii. Repeat 1(b)ii with L1-penalized SVMs

In [35]:
# Standarize
scaler = preprocessing.StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)

In [38]:
# family 

Cs = np.logspace(-5, 8, 10)
param_set = {'C': Cs}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
grid_search_cv = GridSearchCV(l1_svm, param_set , cv=10, scoring = 'accuracy') 
grid_search_cv.fit(scaled_X_train, y_train_family)
best_param = grid_search_cv.best_params_
best_score = grid_search_cv.best_score_
print('Best Parameters: C = %f, Best Score = %f'%(best_param['C'],best_score))


Best Parameters: C = 166.810054, Best Score = 0.935266


In [40]:
pred_l1 = pd.DataFrame()
pred_l1['Family_1'] = grid_search_cv.predict(scaled_X_test)

acc_score = accuracy_score(y_test['Family'], pred_l1['Family_1'])
hamming_l = hamming_loss(y_test['Family'], pred_l1['Family_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9300602130616026
Hamming Loss: 0.0699397869383974


In [41]:
# Genus

Cs = np.logspace(-5, 8, 10)
param_set = {'C': Cs}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
grid_search_cv = GridSearchCV(l1_svm, param_set , cv=10, scoring = 'accuracy') 
grid_search_cv.fit(scaled_X_train, y_train_genus)
best_param = grid_search_cv.best_params_
best_score = grid_search_cv.best_score_
print('Best Parameters: C = %f, Best Score = %f'%(best_param['C'],best_score))

Best Parameters: C = 166.810054, Best Score = 0.957109


In [42]:
pred_l1['Genus_1'] = grid_search_cv.predict(scaled_X_test)

acc_score = accuracy_score(y_test['Genus'], pred_l1['Genus_1'])
hamming_l = hamming_loss(y_test['Genus'], pred_l1['Genus_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9448818897637795
Hamming Loss: 0.05511811023622047


In [43]:
# Species

Cs = np.logspace(-5, 8, 10)
param_set = {'C': Cs}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
grid_search_cv = GridSearchCV(l1_svm, param_set , cv=10, scoring = 'accuracy') 
grid_search_cv.fit(scaled_X_train, y_train_species)
best_param = grid_search_cv.best_params_
best_score = grid_search_cv.best_score_
print('Best Parameters: C = %f, Best Score = %f'%(best_param['C'],best_score))

Best Parameters: C = 5.994843, Best Score = 0.962272


In [45]:
pred_l1['Species_1'] = grid_search_cv.predict(scaled_X_test)

acc_score = accuracy_score(y_test['Species'], pred_l1['Species_1'])
hamming_l = hamming_loss(y_test['Species'], pred_l1['Species_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9490504863362668
Hamming Loss: 0.05094951366373321


<font size = 5> iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance¶

In [47]:
# Family 

sm = SMOTE()
pred_smote = pd.DataFrame()

X_res, y_res = sm.fit_sample(scaled_X_train, y_train['Family'])
X_res_family = pd.DataFrame(X_res)
Y_res_family = pd.DataFrame(y_res)

Cs = np.logspace(-5, 8, 10)
param_set = {'C': Cs}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
grid_search_cv = GridSearchCV(l1_svm, param_set , cv=10, scoring = 'accuracy') 
grid_search_cv.fit(X_res_family, Y_res_family)
best_param = grid_search_cv.best_params_
best_score = grid_search_cv.best_score_
print('Best Parameters: C = %f, Best Score = %f'%(best_param['C'],best_score))

Best Parameters: C = 166.810054, Best Score = 0.953050


In [48]:
pred_smote['Family_1'] = grid_search_cv.predict(scaled_X_test)

acc_score = accuracy_score(y_test['Family'], pred_smote['Family_1'])
hamming_l = hamming_loss(y_test['Family'], pred_smote['Family_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9152385363594256
Hamming Loss: 0.08476146364057434


In [49]:
# Genus 

X_res, y_res = sm.fit_sample(scaled_X_train, y_train['Genus'])
X_res_genus  = pd.DataFrame(X_res)
Y_res_genus  = pd.DataFrame(y_res)

Cs = np.logspace(-5, 8, 10)
param_set = {'C': Cs}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
grid_search_cv = GridSearchCV(l1_svm, param_set , cv=10, scoring = 'accuracy') 
grid_search_cv.fit(X_res_genus, Y_res_genus)
best_param = grid_search_cv.best_params_
best_score = grid_search_cv.best_score_
print('Best Parameters: C = %f, Best Score = %f'%(best_param['C'],best_score))

Best Parameters: C = 166.810054, Best Score = 0.960441


In [50]:
pred_smote['Genus_1'] = grid_search_cv.predict(scaled_X_test)

acc_score = accuracy_score(y_test['Genus'], pred_smote['Genus_1'])
hamming_l = hamming_loss(y_test['Genus'], pred_smote['Genus_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.905048633626679
Hamming Loss: 0.09495136637332098


In [51]:
# Species 

X_res, y_res = sm.fit_sample(scaled_X_train, y_train['Species'])
X_res_species = pd.DataFrame(X_res)
Y_res_species = pd.DataFrame(y_res)

Cs = np.logspace(-5, 8, 10)
param_set = {'C': Cs}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
grid_search_cv = GridSearchCV(l1_svm, param_set , cv=10, scoring = 'accuracy') 
grid_search_cv.fit(X_res_species, Y_res_species)
best_param = grid_search_cv.best_params_
best_score = grid_search_cv.best_score_
print('Best Parameters: C = %f, Best Score = %f'%(best_param['C'],best_score))

Best Parameters: C = 166.810054, Best Score = 0.963792


In [52]:
pred_smote['Species_1'] = grid_search_cv.predict(scaled_X_test)

acc_score = accuracy_score(y_test['Species'], pred_smote['Species_1'])
hamming_l = hamming_loss(y_test['Species'], pred_smote['Species_1'])
print('Accuracy Score:', acc_score)
print('Hamming Loss:', hamming_l)

Accuracy Score: 0.9485873089393237
Hamming Loss: 0.05141269106067624


<font size =5 >Report your conclusions about the classifiers you trained.

(b)ii Gaussian Kernel SVM:

| XXX            | Family               | Genus               | Species            |
|----------------|----------------------|---------------------|--------------------|
| Best C         | 1111.112000          | 1111.112000         | 1111.112000        |
| Best Gamma     | 1.333889             | 1.333889            | 2.000333           |
| Accuracy Score | 0.9898100972672533   | 0.9884205650764243  | 0.9888837424733673 |
| Hamming Loss   | 0.010189902732746642 | 0.01157943492357573 | 0.0111162575266327 |

(b)iii L1-penalized SVMs:

| XXX            | Family             | Genus               | Species             |
|----------------|--------------------|---------------------|---------------------|
| Best C         | 166.810054         | 166.810054          | 5.994843            |
| Best Score     | 0.935266           | 0.957109            | 0.962272            |
| Accuracy Score | 0.9300602130616026 | 0.9448818897637795  | 0.9490504863362668  |
| Hamming Loss   | 0.0699397869383974 | 0.05511811023622047 | 0.05094951366373321 |

(b)iv L1-penalized SVMs with SMOTE:

| XXX            | Family              | Genus               | Species             |
|----------------|---------------------|---------------------|---------------------|
| Best C         | 166.810054          | 166.810054          | 166.810054          |
| Best Score     | 0.953050            | 0.960441            | 0.963792            |
| Accuracy Score | 0.9152385363594256  | 0.905048633626679   | 0.9485873089393237  |
| Hamming Loss   | 0.08476146364057434 | 0.09495136637332098 | 0.05141269106067624 |

## In the above tables , C is the weight of the SVM parameter and gamma is the width of the Gaussian Kernel. 
## The SVM using Gaussian Kernels had the best the Accuracy Score and Hamming Loss. 
## So, this non-linear SVM performed better than the L1-penalized SVMs. Using SMOTE to remedy class imbalance did not improve the L1-penalized SVM model. Accordint to the Accuracy Scores and Hamming Loss for the three classifiers trained, the dataset is probably not linearly separated and was able to fit better with a non-linear model.

--------------------------------------------------------------------------------------------------------
<font size = 5>2. K-Means Clustering on a Multi-Class and Multi-Label Data Set Monte-Carlo Simulation: Perform the following procedures 50 times, and report
the average and standard deviation of the 50 Hamming Distances that you calculate.

<font size = 5>(a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know. 

(b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.


(c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss5 between the true labels and the labels assigned by clusters.

In [54]:
def kmeans_cluster():
    
    cluster_df = pd.read_csv('Frogs_MFCCs.csv')
    X_df = cluster_df.iloc[:,:-4]
    k_list = list(range(2,51)) 
    k_scores_ch = []
    for k in k_list: 
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(X_df)  
        k_labels = kmeans.labels_
        CH = calinski_harabaz_score(X_df, k_labels)
        k_scores_ch.append(CH)

    opt_k_ch = k_list[k_scores_ch.index(max(k_scores_ch))]
    kmeans = KMeans(n_clusters=opt_k_ch).fit(X_df)
    cluster_df['cluster'] = kmeans.labels_

    for c in range(0,opt_k_ch):
        family = cluster_df[cluster_df.cluster == c]['Family'].value_counts().index[0]
        cluster_df.loc[cluster_df.cluster == c, 'Family_Majority'] = family
        genus = cluster_df[cluster_df.cluster == c]['Genus'].value_counts().index[0]
        cluster_df.loc[cluster_df.cluster == c, 'Genus_Majority'] = genus
        species = cluster_df[cluster_df.cluster == c]['Species'].value_counts().index[0]
        cluster_df.loc[cluster_df.cluster == c, 'Species_Majority'] = species
    score_list = []
    
    labels = ['Family', 'Genus', 'Species']

    for c in range(0,opt_k_ch):
        
        cluster_score_list = []
        cluster = cluster_df[cluster_df.cluster == c]
        for l in labels:
            cluster_label = l + '_Majority'

            score = hamming_loss(cluster[l], cluster[cluster_label])
            cluster_score_list.append(score)
            print('Cluster %d >> %s: %f'%(c,l,score))
            
        avg_cluster_loss = np.mean(cluster_score_list)
        score_list.append(avg_cluster_loss)
        print('Average for Cluster %d: %f'%(c,avg_cluster_loss))
        
    average_loss = np.mean(score_list)
    print('Average for iteration:',average_loss)
    return(average_loss)

In [56]:
hamming_loss_list = []

for i in range(1,51):
    
    print('Iteration Number -> %d : '%(i))
    
    hamming_l = kmeans_cluster()
    hamming_loss_list.append(hamming_l)
    
    print('------------------------------------------')

Iteration Number -> 1 : 
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.035575
Cluster 0 >> Species: 0.035575
Average for Cluster 0: 0.035205
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Average for iteration: 0.2985772581674484
------------------------------------------
Iteration Number -> 2 : 
Cluster 0 >> Family: 0.432583
Cluster 0 >> Genus: 0.561023
Cluster 0 >> Species: 0.692244
Average for Cluster 0: 0.561950
Cluster 1 >> Family: 0.034464
Cluster 1 >> Genus: 0.035575
Cluster 1 >> Species: 0.035575
Average for Cluster 1: 0.035205
Average for iteration: 0.2985772581674484
------------------------------------------
Iteration Number -> 3 : 
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.035575
Cluster 0 >> Species: 0.035575
Average for Cluster 0: 0.035205
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Average for iteration: 

Cluster 0 >> Family: 0.432583
Cluster 0 >> Genus: 0.561023
Cluster 0 >> Species: 0.692244
Average for Cluster 0: 0.561950
Cluster 1 >> Family: 0.034464
Cluster 1 >> Genus: 0.035575
Cluster 1 >> Species: 0.035575
Average for Cluster 1: 0.035205
Average for iteration: 0.2985772581674484
------------------------------------------
Iteration Number -> 26 : 
Cluster 0 >> Family: 0.432583
Cluster 0 >> Genus: 0.561023
Cluster 0 >> Species: 0.692244
Average for Cluster 0: 0.561950
Cluster 1 >> Family: 0.034464
Cluster 1 >> Genus: 0.035575
Cluster 1 >> Species: 0.035575
Average for Cluster 1: 0.035205
Average for iteration: 0.2985772581674484
------------------------------------------
Iteration Number -> 27 : 
Cluster 0 >> Family: 0.432583
Cluster 0 >> Genus: 0.561023
Cluster 0 >> Species: 0.692244
Average for Cluster 0: 0.561950
Cluster 1 >> Family: 0.034464
Cluster 1 >> Genus: 0.035575
Cluster 1 >> Species: 0.035575
Average for Cluster 1: 0.035205
Average for iteration: 0.2985772581674484
----

Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.035575
Cluster 0 >> Species: 0.035575
Average for Cluster 0: 0.035205
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Average for iteration: 0.2985772581674484
------------------------------------------
Iteration Number -> 50 : 
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.035575
Cluster 0 >> Species: 0.035575
Average for Cluster 0: 0.035205
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Average for iteration: 0.2985772581674484
------------------------------------------


In [58]:
print('the best K is 2 selected using CH in above 50 iterations.')
hamming_std = np.std(np.array(hamming_loss_list))
hamming_avg = np.mean(np.array(hamming_loss_list))
print('Average hamming loss in all iterations:', hamming_avg)
print('STD hamming loss in all iterations:', hamming_std)

the best K is 2 selected using CH in above 50 iterations.
Average hamming loss in all iterations: 0.2985772581674484
STD hamming loss in all iterations: 0.0


3. ISLR 10.7.2 : Suppose that we have four observations, for which we compute a dissimilarity matrix, given by [] For instance, the dissimilarity between the first and second obser- vations is 0.3, and the dissimilarity between the second and fourth observations is 0.8.
(a) On the basis of this dissimilarity matrix, sketch the dendrogram that results from hierarchically clustering these four observa- tions using complete linkage. Be sure to indicate on the plot the height at which each fusion occurs, as well as the observations corresponding to each leaf in the dendrogram.

Step 1. We already have

In [60]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://i.imgur.com/oVhhRal.png")

Step 2. i=4 : We may see that 0.3 is the minimum dissimilarity, so we fuse observations 1 and 2 to form cluster (1,2) at hight 0.3. We now have the new dissimilarity matrix

In [61]:
Image(url="https://imgur.com/FcPRAny.png")

i=3  : We now see that the minimum dissimilarity is 0.45, so we fuse observations 3 and 4 to form cluster (3,4) at height 0.45. We now have the new dissimilarity matrix

In [62]:
Image(url="https://imgur.com/VaNpWcb.png")

i=4  : It remains to fuse clusters (1,2) and (3,4) to form cluster ((1,2),(3,4)) at height 0.8.

In [64]:
Image(url="https://imgur.com/2AJEAD7.png")

(b) Repeat (a), this time using single linkage clustering.

In [65]:
Image(url="https://i.imgur.com/oVhhRal.png")

## Step 2: i=4 : We see that 0.3 is the minimum dissimilarity, so we fuse observations 1 and 2 to form cluster (1,2) at hight 0.3. We now have the new dissimilarity matrix

In [66]:
Image(url="https://imgur.com/OszUoBX.png")

## i=3  : Now we see that the minimum dissimilarity is 0.4, so we fuse cluster (1,2) and observation 3 to form cluster ((1,2),3) at height 0.4. We now have the new dissimilarity matrix

In [68]:
Image(url="https://imgur.com/MyAu9bI.png")

## i=4 : It remains to fuse clusters ((1,2),3) and observation 4 to form cluster (((1,2),3),4) at height 0.45.

In [70]:
Image(url="https://imgur.com/MqptR5y.png")

(c) Suppose that we cut the dendogram obtained in (a) such that two clusters result. Which observations are in each cluster?

## In this case, we have clusters (1,2) and (3,4).

(d) Suppose that we cut the dendogram obtained in (b) such that two clusters result. Which observations are in each cluster?

## In this case, we have clusters ((1,2),3) and (4).

(e) It is mentioned in the chapter that at each fusion in the den- drogram, the position of the two clusters being fused can be swapped without changing the meaning of the dendrogram. Draw a dendrogram that is equivalent to the dendrogram in (a), for which two or more of the leaves are repositioned, but for which the meaning of the dendrogram is the same.

In [71]:
Image(url="https://imgur.com/ty71XHn.png")